# 모델 다운로드

In [ ]:
#!pip install -U git+https://github.com/leondgarse/keras_efficientnet_v2

# GPU 버전 체크

In [44]:
!nvidia-smi

Thu Sep  9 20:10:09 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 471.11       Driver Version: 471.11       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
|  0%   43C    P8    21W / 250W |  11089MiB / 11264MiB |     15%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 모듈 Import

In [47]:
import tensorflow as tf
import pandas as pd
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import keras_efficientnet_v2 as efficientnet_v2

from tensorflow.keras.layers import Dense, Input, Conv2D, Dropout, Flatten, Activation, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Model, Sequential
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow import keras
from tensorflow import keras
from skimage.transform import resize
from PIL import Image
import cv2
import numpy as np
import glob
from tqdm import tqdm

# Configuration

In [48]:
seed = 42
batch_size = 64
width = 240
height = 240
epochs = 30
NUM_TRAIN = 0
NUM_TEST = 0
dropout_rate = 0.2
input_shape = (height, width, 3)
num_classes = 322

In [49]:
print(tf.__version__)
tf.random.set_seed(seed)

2.6.0


## Read DataFrame
미리 저장된 데이터 프레임을 읽어옵니다.

In [50]:
df = pd.read_json('./data/dataset/car.json', "r", encoding='UTF8')
df = df.T.rename_axis('class_name').reset_index()

<ipython-input-50-4bb5d48c4a0b>:1: FutureWarning: Starting with Pandas version 2.0 all arguments of read_json except for the argument 'path_or_buf' will be keyword-only
  df = pd.read_json('./data/dataset/car.json', "r", encoding='UTF8')


# DataFrame Split

In [51]:
def train_test_split_custom(df, train_size, random_state=42):
    train_size = 0.8

    df_train = pd.DataFrame(data={'class_name':[]})
    df_test = pd.DataFrame(data={'class_name':[]})

    count_train = 0
    count_test = 0
    for index, class_name in enumerate(df['class_name']) : 
        image_paths = df['image_path'].iloc[index]
        random.Random(random_state).shuffle(image_paths)
        image_paths_train = image_paths[: round(len(image_paths) * train_size)]
        image_paths_test = image_paths[round(len(image_paths) * train_size):]

        for path in image_paths_train :
            df_train.loc[path] = df['class_name'].iloc[index]
        for path in image_paths_test :
            df_test.loc[path] = df['class_name'].iloc[index] 

    df_train = df_train.reset_index().rename(columns={"index": "path"})
    df_test = df_test.reset_index().rename(columns={"index": "path"})
    return df_train, df_test

In [52]:
df_train , df_valid = train_test_split_custom(df, train_size=0.8, random_state=seed)

In [53]:
df_train.tail()

,path,class_name
25755,./data/dataset/혼다_세단_어코드_2018-2020/세단_어코드-212_...,혼다/세단_어코드/2018-2020
25756,./data/dataset/혼다_세단_어코드_2018-2020/세단_어코드-212.jpg,혼다/세단_어코드/2018-2020
25757,./data/dataset/혼다_세단_어코드_2018-2020/세단_어코드-130_...,혼다/세단_어코드/2018-2020
25758,./data/dataset/혼다_세단_어코드_2018-2020/세단_어코드-184_...,혼다/세단_어코드/2018-2020
25759,./data/dataset/혼다_세단_어코드_2018-2020/세단_어코드-154_...,혼다/세단_어코드/2018-2020


In [54]:
df_valid.tail()

,path,class_name
6435,./data/dataset/혼다_세단_어코드_2018-2020/세단_어코드-239.jpg,혼다/세단_어코드/2018-2020
6436,./data/dataset/혼다_세단_어코드_2018-2020/세단_어코드-226_...,혼다/세단_어코드/2018-2020
6437,./data/dataset/혼다_세단_어코드_2018-2020/세단_어코드-115.jpg,혼다/세단_어코드/2018-2020
6438,./data/dataset/혼다_세단_어코드_2018-2020/세단_어코드-16.jpg,혼다/세단_어코드/2018-2020
6439,./data/dataset/혼다_세단_어코드_2018-2020/세단_어코드-177_...,혼다/세단_어코드/2018-2020


In [55]:
NUM_TRAIN = len(df_train)
NUM_TEST = len(df_valid)

print(f'number of train : {NUM_TRAIN}\nnumber of validation : {NUM_TEST}')

number of train : 25760
number of validation : 6440


# ImageDataGenerator

In [56]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
        df_train,
        x_col = 'path',
        y_col = 'class_name',
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_dataframe(
        df_valid,
        x_col = 'path',
        y_col = 'class_name',
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='categorical')

Found 25760 validated image filenames belonging to 322 classes.
Found 6440 validated image filenames belonging to 322 classes.


# Model Create 함수

In [57]:
def create_model(version = 'v1', model_type='b0', input_shape= (224,224,3), n_classes = 322):
  if version == 'v1':
    if model_type == 'b0':
      base_model = efficientnet_v2.EfficientNetV1(model_type, input_shape=input_shape, num_classes=0, include_preprocessing=False,  pretrained='imagenet')
    elif model_type == 'b1':
      base_model = efficientnet_v2.EfficientNetV1(model_type, input_shape=input_shape, num_classes=0,include_preprocessing=False, pretrained='imagenet')
  
  elif version == 'v2':
    if model_type == 'b0':
      base_model = efficientnet_v2.EfficientNetV2(model_type, input_shape=input_shape, num_classes=0, include_preprocessing=False, pretrained='imagenet')
    elif model_type == 'b1':
      base_model = efficientnet_v2.EfficientNetV2(model_type, input_shape=input_shape, num_classes=0, include_preprocessing=False, pretrained='imagenet')
      
  model = Sequential()
  model.add(Input(shape=input_shape))
  model.add(base_model)
  model.add(GlobalAveragePooling2D())
  model.add(Dropout(0.2))
  model.add(Dense(n_classes, activation='softmax'))
  

  return model

# 콜백함수 정의

In [58]:
# callback
modelname = f'./data/checkpoint-epoch-{epochs}-batch-{batch_size}-trial-005.h5'

checkpoint = ModelCheckpoint(modelname,
                             monitor='val_loss',
                             save_best_only=True,
                             mode='auto'
                            )

reduceLR = ReduceLROnPlateau(monitor='val_loss',
                             factor=0.5,
                             patience=2
                             )

earlystopping = EarlyStopping(monitor='val_loss',
                              patience=5,
                             )

# (선택1-1) Model 생성

In [ ]:
# model = create_model(version = 'v2', model_type='b1', input_shape= input_shape, n_classes = num_classes)

# model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['acc'])
# model.summary()

## model 학습

In [ ]:
# history = model.fit(train_generator, 
#                  steps_per_epoch=NUM_TRAIN/batch_size,
#                  validation_data=validation_generator, 
#                  validation_steps=NUM_TEST/batch_size,
#                  epochs=epochs,
#                  callbacks = [checkpoint, earlystopping],
#                  verbose = 1)

## model 학습률 그래프

In [ ]:
# import matplotlib.pyplot as plt

# #Plotting 
# acc = history.history['acc'] 
# val_acc = history.history['val_acc'] 
# loss = history.history['loss'] 
# val_loss = history.history['val_loss'] 

# plt.figure(figsize=(8, 8)) 

# plt.subplot(2, 1, 1) 
# plt.plot(acc, label='Training Accuracy') 
# plt.plot(val_acc, label='Validation Accuracy') 
# plt.legend(loc='lower right') 
# plt.ylabel('Accuracy') 
# plt.ylim([min(plt.ylim()),1]) 
# plt.title('Training and Validation Accuracy') 

# plt.subplot(2, 1, 2) 
# plt.plot(loss, label='Training Loss') 
# plt.plot(val_loss, label='Validation Loss') 
# plt.legend(loc='upper right') 
# plt.ylabel('Cross Entropy') 
# plt.ylim([0,10]) 
# plt.title('Training and Validation Loss') 
# plt.xlabel('epoch') 
# plt.show()

## Model Save

In [ ]:
# savemodel = model
# filepath = './data/v2_b1_model_240x240.h5'
# keras.models.save_model(model = savemodel, filepath=filepath)

# (선택1-2) 학습된 Model 불러오기

In [59]:
model_path = './data/[4]input 96600, model B0, inputsize (240,240), freeze 0, batchsize 64, epoch 30.h5'
model = keras.models.load_model(model_path)
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
EfficientNetV2 (Functional)  (None, 8, 8, 1280)        5919312   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 322)               412482    
Total params: 6,331,794
Trainable params: 6,271,186
Non-trainable params: 60,608
_________________________________________________________________


# 히트맵

In [60]:
def make_gradcam_heatmap(img_array, model, pre_trained,last_conv_layer_name, classifier_layer_names):
    
    # First, we create a model that maps the input image to the activations
    # of the last conv layer
    last_conv_layer  = model.get_layer(pre_trained).get_layer(last_conv_layer_name)
    conv_model       = keras.Model(model.get_layer(pre_trained).inputs, last_conv_layer.output)
    # Second, we create a model that maps the activations of the last conv
    # layer to the final class predictions
    classifier_input = keras.Input(shape=last_conv_layer.output.shape[1:])
    x = classifier_input
    
    for layer_name in classifier_layer_names:
        x = model.get_layer(layer_name)(x)

    classifier_model = keras.Model(classifier_input, x)
    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer  
    with tf.GradientTape() as tape:
        # Compute activations of the last conv layer and make the tape watch it
        last_conv_layer_output = conv_model(img_array)
        tape.watch(last_conv_layer_output)
        
        # Compute class predictions
        preds = classifier_model(last_conv_layer_output)
        top_pred_index = tf.argmax(preds[0])
        top_class_channel = preds[:, top_pred_index]

    # This is the gradient of the top predicted class with regard to
    # the output feature map of the last conv layer
    grads = tape.gradient(top_class_channel, last_conv_layer_output)
    
    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    last_conv_layer_output = last_conv_layer_output.numpy()[0]

    # is our saliency heatmap of class activation
    saliency = np.mean(last_conv_layer_output, axis=-1)
    saliency = np.maximum(saliency, 0) / np.max(saliency)

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    pooled_grads = pooled_grads.numpy()
    
    for i in range(pooled_grads.shape[-1]):
        last_conv_layer_output[:, :, i] *= pooled_grads[i]

    # The channel-wise mean of the resulting feature map
    # is our grad_cam heatmap of class activation
    grad_cam = np.mean(last_conv_layer_output, axis=-1)
    grad_cam = np.maximum(grad_cam, 0) / np.max(grad_cam)

    return grad_cam, saliency

In [61]:
def merge_with_heatmap(original_img, heatmap):
    original_img = np.array(original_img)
    resized_heatmap=resize(heatmap, (240, 240))
    resized_heatmap = np.uint8(255*resized_heatmap)
    resized_heatmap = cv2.applyColorMap(resized_heatmap, cv2.COLORMAP_JET)
    #resized_heatmap = cv2.cvtColor(resized_heatmap, cv2.COLOR_RGB2BGR)
    return cv2.addWeighted(resized_heatmap, 0.7, original_img, 0.5, 6)

def convert_to_heatmap(heatmap):
    heatmap = np.uint8(255*heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    return cv2.cvtColor(heatmap, cv2.COLOR_RGB2BGR)

In [62]:
def show_hotmap (img, heatmap, title='Heatmap', alpha=0.6, cmap='jet', axisOnOff='off'):
    '''
    img     :    Image
    heatmap :    2d narray
    '''
    resized_heatmap=resize(heatmap, img.size)
    
    fig, ax = plt.subplots()
    ax.imshow(img)
    ax.imshow(resized_heatmap, alpha=alpha, cmap=cmap)
    plt.axis(axisOnOff)
    plt.title(title)
    plt.show()

# 테스트

In [63]:
def prepare_single_input(img_path, target_size=(224, 224)):
    img = image.load_img(img_path, target_size=target_size)
    img = image.img_to_array(img)
    img /= 255.
    img = np.expand_dims(img, axis= 0) # (1, 224, 224, 3)
    return img

In [76]:
def predict_image(Mymodel, img_path, top_k_num = 3):
    image = prepare_single_input(img_path, target_size = (240, 240))
    result = Mymodel.predict([image])[0]

    result = list(result)
    
    classname_list = []
    pred_value_list = []
    for _ in range(top_k_num) :
      index= result.index(max(result))
      classname_list.append(classes_dict[index])
      pred_value_list.append(max(result))
      result[index] = 0.

    return classname_list, pred_value_list

In [65]:
classes_dict = validation_generator.class_indices
classes_dict = dict(map(reversed, classes_dict.items()))

In [66]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
EfficientNetV2 (Functional)  (None, 8, 8, 1280)        5919312   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 322)               412482    
Total params: 6,331,794
Trainable params: 6,271,186
Non-trainable params: 60,608
_________________________________________________________________


In [77]:
######################################################################################
#defalut classifier_layer_names #기본 classifier_layer입니다.
#classifier_layer_names =  ['global_average_pooling2d', 'dropout', 'dense']
classifier_layers =  model.layers[-3:]
classifier_layer_names = []
for layer in classifier_layers:
    classifier_layer_names.append(layer.name)
last_conv_layer_name   = 'post_swish'
pre_train= 'EfficientNetV2'
Top_K = 1

true_cnt = 0
false_cnt = 0

#####################################################################################
image_paths = glob.glob('./data/test/dataset/**/*.jpg')
for path in tqdm(image_paths, total = len(image_paths)) :
    img = Image.open(path).resize(size=input_shape[:2])  
    img_array = prepare_single_input(path, input_shape[:2])
    #test
    grad_cam, saliency = make_gradcam_heatmap(img_array, model, pre_train,last_conv_layer_name, classifier_layer_names)
    pred_classnames, pred_values = predict_image(model, path)


    grad_cam_merge = merge_with_heatmap(img, grad_cam)
    saliency_merge = merge_with_heatmap(img, saliency)

    grad_cam = convert_to_heatmap(grad_cam)
    saliency = convert_to_heatmap(saliency)

    #save

    #./data/test/dataset\BMW_SUV_X5_2019-2020\MAH02945_1017.jpg1.jpg -> BMW_SUV_X5_2019-2020
    true_classname = path.split('\\')[1]

    for index, pred_classname in enumerate(pred_classnames) :
        #BMW/SUV_X5/2019-2020 -> BMW_SUV_X5_2019-2020 True_class와 맞추기 위해서.
        pred_classnames[index] = pred_classname.replace('/','_')

    #pred_values안에 0.1이 있다면 True 그렇지 않다면 False인 리스트 생성
    flag_pred_value = list(map(lambda x : True if x >= 0.1 else False, pred_values))

    #실제 클래스와 예측 클래스가 일치하면, 그리고 predict_value가 0.1이상이라면 실행
    #Select save path True or False
    if (true_classname in pred_classnames) & (True in flag_pred_value) : 
        true_cnt+=1
        #./data/test/dataset\BMW_SUV_X5_2019-2020\MAH02945_1017.jpg1.jpg -> ./data/test/result/true/MAH02945_1017.jpg1.jpg
        split_paths = path.split('\\')
        save_path = split_paths[0].replace('dataset','result') +'/' + 'true' + '/' + split_paths[-1]
    else : 
        false_cnt+=1
        #./data/test/dataset\BMW_SUV_X5_2019-2020\MAH02945_1017.jpg1.jpg -> ./data/test/result/false/MAH02945_1017.jpg1.jpg
        split_paths = path.split('\\')
        save_path = split_paths[0].replace('dataset','result') +'/' + 'false' + '/' + split_paths[-1]

    #Save original image 
    img.save(save_path)
    #./data/test_result/true/MAH02945_1017.jpg1.jpg -> ./data/test_result/true or false/MAH02945_1017.jpg1.txt
    tmp_path = save_path[:-3] + 'txt'
    with open(tmp_path, "w", encoding="UTF-8") as f:
        f.write(f"[실제 클래스]\n")
        f.write(f"{true_classname}\n")
        f.write(f"[예측 클래스 Top 3]\n")
        f.write(f"{pred_classnames[0]}, 예측률 : {pred_values[0]}\n")
        f.write(f"{pred_classnames[1]}, 예측률 : {pred_values[1]}\n")
        f.write(f"{pred_classnames[2]}, 예측률 : {pred_values[2]}\n")

    #Save grad_cam image
    #./data/test_result/true/MAH02945_1017.jpg1.jpg -> ./data/test_result/true or false/MAH02945_1017.jpg1_gc.jpg
    tmp_path = save_path[:-4] + '_gc.jpg'
    plt.figure(figsize=(input_shape[0] / 100, input_shape[0] / 100))
    plt.imshow(grad_cam)
    plt.axis('off'), plt.xticks([]), plt.yticks([])
    plt.tight_layout()
    plt.subplots_adjust(left = 0, bottom = 0, right = 1, top = 1, hspace = 0, wspace = 0)
    plt.savefig(tmp_path, bbox_inces='tight', pad_inches=0, dpi=100)
    plt.close()
    

    #Save saliency image
    #./data/test_result/true/MAH02945_1017.jpg1.jpg -> ./data/test_result/true or false/MAH02945_1017.jpg1_sa.jpg
    tmp_path = save_path[:-4] + '_sa.jpg'
    plt.figure(figsize=(input_shape[0] / 100, input_shape[0] / 100))
    plt.imshow(saliency)
    plt.axis('off'), plt.xticks([]), plt.yticks([])
    plt.tight_layout()
    plt.subplots_adjust(left = 0, bottom = 0, right = 1, top = 1, hspace = 0, wspace = 0)
    plt.savefig(tmp_path, bbox_inces='tight', pad_inches=0, dpi=100)
    plt.close()
    

    #Save grad_cam-Merge image
    #./data/test_result/true/MAH02945_1017.jpg1.jpg -> ./data/test_result/true or false/MAH02945_1017.jpg1_gcm.jpg
    tmp_path = save_path[:-4] + '_gcm.jpg'
    cv2.imwrite(tmp_path, grad_cam_merge)

    #Save saliency-Merge image
    #./data/test_result/true/MAH02945_1017.jpg1.jpg -> ./data/test_result/true or false/MAH02945_1017.jpg1_sam.jpg
    tmp_path = save_path[:-4] + '_sam.jpg'
    cv2.imwrite(tmp_path, saliency_merge)

100%|██████████| 836/836 [04:04<00:00,  3.42it/s]


In [ ]:
acc = true_cnt / (true_cnt+false_cnt)
print(f"Top{Top_K} Acc : {round(acc,2)}")